In [6]:
%matplotlib notebook
from ipywidgets import *
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
from ipywidgets import interact

PS = pd.read_csv('P-S_results.csv')
x_data = np.array(PS['mPAP'])
y_Lit = np.array(PS['t_vortex (lit)'])
y_CFD = np.array(PS['t_vortex (CFD)'])
y_4D = np.array(PS['t_vortex (4D)'])
y_data = [y_Lit, y_CFD, y_4D]

In [12]:
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import ipywidgets as widgets
from ipywidgets import interact

def interactive_selection_plot(x_data, y_data_sets, methods):
    """
    Creates an interactive plot with multiple vortex duration sets, allowing users to select which method to display.

    Args:
        x_data: Pressure values (mPAP, in mmHg)
        y_data_sets: A list of lists or arrays, each representing a method of vortex duration quantification.
    """

    def update_plot(**kwargs):
        """
        Updates the plot with the selected y-data sets and regression lines.

        Args:
          kwargs: A dictionary of checkbox names (e.g., 'checkbox_1') and their corresponding boolean values.
        """
        # Create a Plotly figure
        fig = go.Figure()

        for i, (checkbox_name, selected) in enumerate(kwargs.items()):
            if selected:
                label = methods[i]
                fig.add_trace(go.Scatter(x=x_data, y=y_data_sets[i], mode='markers', name=label))
                
                # Add regression line
                x_data_PH = x_data[10:] - 16
                y_data_PH = y_data_sets[i][10:]
                y_data_true = y_data_sets[0][10:]
                
                # Perform linear regression
                reg = LinearRegression(fit_intercept=False).fit(x_data_PH.reshape(-1, 1), y_data_PH.reshape(-1, 1))
                y_reg = reg.predict(x_data_PH.reshape(-1, 1))  # Get the predicted y-values for the regression line
                r_squared = r2_score(y_data_PH, y_reg)

                # Add regression line
                fig.add_trace(go.Scatter(x=x_data_PH+16, y=y_reg.flatten(), mode='lines', 
                                         name=f'{label} Regression, R2 = {r_squared:.2f}', line=dict(dash='dash')))

                
                '''reg = LinearRegression(fit_intercept=False).fit(x_data_PH.reshape(-1, 1), y_data_PH.reshape(-1, 1))
                score = reg.coef_ * np.transpose(x_data_PH)
                r_squared = r2_score(y_data_true.reshape(-1, 1), y_data_PH.reshape(-1, 1))

                fig.add_trace(go.Scatter(x=x_data_PH + 16, y=score, mode='lines', 
                                         name=f'{label} Regression, R2 = {r_squared:.2f}', line=dict(dash='dash')))
'''
        # Update layout
        fig.update_layout(
            title='Interactive Selection Plot with Regression',
            xaxis_title='mPAP (mmHg)',
            yaxis_title='Vortex Duration',
            template='plotly_white',
            showlegend=True
        )

        fig.show()

    # Create checkboxes for each data set
    checkboxes = {}
    for i, y_data_set in enumerate(y_data_sets):
        checkbox = widgets.Checkbox(value=True, description=str(methods[i]))
        checkboxes[f'checkbox_{i+1}'] = checkbox

    # Create the interactive plot
    interact(update_plot, **checkboxes)


methods = ['Literature', 'CFD', '4D']
interactive_selection_plot(x_data, y_data, methods)


interactive(children=(Checkbox(value=True, description='Literature'), Checkbox(value=True, description='CFD'),…

In [14]:
import plotly.graph_objects as go
import numpy as np
from sklearn.linear_model import LinearRegression
from sklearn.metrics import r2_score
import ipywidgets as widgets
from ipywidgets import interact

def interactive_correspondence_plot(x_data, y_data_sets, methods):
    """
    Creates an interactive plot with correspondence between literature predicted vortex duration and either CFD or 4D methods,
    allowing users to select which method to display.

    Args:
        x_data: Literature predicted vortex duration
        y_data_sets: A list of lists or arrays, each representing a method of vortex duration quantification.
    """

    def update_plot(**kwargs):
        """
        Updates the plot with the selected y-data sets and regression lines.

        Args:
          kwargs: A dictionary of checkbox names (e.g., 'checkbox_1') and their corresponding boolean values.
        """
        # Create a Plotly figure
        fig = go.Figure()

        for i, (checkbox_name, selected) in enumerate(kwargs.items()):
            if selected:
                label = methods[i]
                
                # Plot the data points
                fig.add_trace(go.Scatter(x=x_data, y=y_data_sets[i], mode='markers', name=label))

                # Perform linear regression
                reg = LinearRegression().fit(x_data.reshape(-1, 1), y_data_sets[i].reshape(-1, 1))
                y_reg = reg.predict(x_data.reshape(-1, 1))  # Predicted y values for the regression line
                r_squared = r2_score(y_data_sets[i], y_reg)

                # Add regression line
                fig.add_trace(go.Scatter(x=x_data, y=y_reg.flatten(), mode='lines', 
                                         name=f'{label} Regression, R2 = {r_squared:.2f}', line=dict(dash='dash')))

        # Update layout
        fig.update_layout(
            title='Interactive Correspondence Plot with Regression',
            xaxis_title='Literature Vortex Duration',
            yaxis_title='Method Vortex Duration',
            template='plotly_white',
            showlegend=True
        )

        fig.show()

    # Create checkboxes for each data set
    checkboxes = {}
    for i, y_data_set in enumerate(y_data_sets):
        checkbox = widgets.Checkbox(value=True, description=str(methods[i]))
        checkboxes[f'checkbox_{i+1}'] = checkbox

    # Create the interactive plot
    interact(update_plot, **checkboxes)

new_y_data = [y_CFD, y_4D]
methods = ['CFD', '4D']

interactive_correspondence_plot(y_Lit, new_y_data, methods)



interactive(children=(Checkbox(value=True, description='CFD'), Checkbox(value=True, description='4D'), Output(…